In [156]:
# 加载需要的包
import pandas as pd
import numpy as np
import datetime
from typing import Union

In [157]:
# 连接wind
from WindPy import w
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [158]:
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Border, Side, PatternFill, Alignment
import os

class Exl:
    # 生成新的 Excel 文件并写入多个 DataFrame 数据
    @staticmethod
    def generate_excel(file_path, df_dict):
        wb = Workbook()
        
        for sheet_name, df in df_dict.items():
            if sheet_name in wb.sheetnames:
                ws = wb[sheet_name]
            else:
                ws = wb.create_sheet(title=sheet_name)
            
            # 写入列名
            for col_num, column_title in enumerate(df.columns, 1):
                ws.cell(row=1, column=col_num, value=column_title)
            
            # 写入数据
            for row_num, row_data in enumerate(df.values, 2):
                for col_num, cell_value in enumerate(row_data, 1):
                    ws.cell(row=row_num, column=col_num, value=cell_value)
        
        # 删除默认创建的第一个工作表
        if 'Sheet' in wb.sheetnames:
            wb.remove(wb['Sheet'])
        
        # 保存初始 Excel 文件
        wb.save(file_path)
        # 保存初始 Excel 文件
        wb.save(file_path)
    @staticmethod
    def excel_style_1(file_path, number_format='0.0000', columnC_format='yyyy-mm-dd'):
        # 加载Excel文件
        wb = load_workbook(file_path)
        for sheet in wb.sheetnames:
            ws = wb[sheet]
    
            # 设置字体
            font = Font(name='等线', size=10, color='FF333333')
            font_header = Font(name='等线', size=10, color='FFffffff', bold=True, italic=False)
            
            # 设置边框(Light Lilac)
            thin = Side(style='thin', color='FFd5d9eb')
            thick = Side(style='thick', color='FFd5d9eb')
            border = Border(left=thin, right=thin, top=thin, bottom=thin)               # 表格边框
            border_header = Border(left=thin, right=thin, top=thin, bottom=thick)       # 表头边框
            
            # 设置填充(Light Blue)
            fill_header = PatternFill(start_color='FF00a1e2', end_color='FF00a1e2', fill_type='solid')
            
            # 设置对齐
            alignment = Alignment(horizontal='center', vertical='center')
            
            # 遍历所有单元格, 应用样式
            for row in ws.iter_rows():
                for cell in row:
                    cell.font = font
                    cell.border = border
                    cell.alignment = alignment
                    cell.number_format = number_format
            
            # 遍历首行, 应用样式
            for cell in ws[1]:
                cell.font = font_header
                cell.border = border_header
                cell.fill = fill_header
                
            # 遍历首列, 应用样式
            if sheet == '产品信息':
                for cell in ws['C']:
                    cell.number_format = columnC_format
            
            # 冻结首行首列
            ws.freeze_panes = 'B2'
            
            # 遍历所有列字母, 调整列宽
            for col in ws.iter_cols():
                column_letter = get_column_letter(col[0].column)
                ws.column_dimensions[column_letter].width = 15
        
        # 保存Excel
        wb.save(file_path)

In [159]:
def convert_dates(date: Union[None, str, datetime.date, datetime.datetime]) -> datetime.date:
    if date is None:
        today = datetime.date.today()
        return datetime.date(today.year + 100, today.month, today.day)
    elif isinstance(date, str):
        return datetime.datetime.strptime(date, "%Y-%m-%d").date()
    elif isinstance(date, datetime.datetime):
        return date.date()
    elif isinstance(date, datetime.date):
        return date
    else:
        raise Exception(f"Unrecognized date {date} with {type(date)} !!")

In [160]:
def get_dates_between_period(
        start_date: Union[str, datetime.date],
        end_date: Union[str, datetime.date],
        freq: str = 'D'
) -> list:
        date_range = pd.date_range(start_date, end_date, freq=freq).to_list()
        date_range = [convert_dates(i) for i in date_range]

        return date_range



In [161]:
# 参数设置
trade_date = '20240630'
start_date = '2021-09-30'
end_date = '2024-06-30' 
quarters = get_dates_between_period(start_date,end_date,freq='3M')

In [162]:
# 数据读取
fof_data = pd.read_excel('./data/基金代码7-10.xlsx').fillna(0)
fund_code = fof_data['基金代码'].to_list()

2.产品规模权重

In [163]:
# 从Wind中获取基金公司和业绩数据
fund_return = w.wss(fund_code,"return_1y, return_2y, return_3y","annualized=0;tradeDate=" + trade_date).Data
fund_mgrcomp = w.wss(fund_code, "fund_mgrcomp, fund_setupdate, sec_name").Data
return_df = pd.DataFrame({'基金代码': fund_code, '基金简称':fund_mgrcomp[2],'成立日期':fund_mgrcomp[1], '基金公司': fund_mgrcomp[0],'近一年回报': fund_return[0], '近两年回报': fund_return[1], '近三年回报': fund_return[2]})

In [164]:
# 从Wind中获取规模数据
scale_data = {}
scale_data['基金代码'] = fund_code
scale_data['基金简称'] = fund_mgrcomp[2]
scale_data['基金公司'] = fund_mgrcomp[0]
scale_data['成立日期'] = fund_mgrcomp[1]
for date in quarters:
    wind_data = w.wss(fund_code, "netasset_total", f"unit=1;tradeDate={date.strftime('%Y-%m-%d')};").Data[0]
    wind_data = [round(x / 1e8, 2) if isinstance(x, (int, float)) and x is not None else x for x in wind_data]
    scale_data[date.strftime('%Y%m%d')] = wind_data

In [165]:
fund_scale_df = pd.DataFrame(scale_data)

In [166]:
fund_scale_df

,基金代码,基金简称,基金公司,成立日期,20210930,20211231,20220331,20220630,20220930,20221231,20230331,20230630,20230930,20231231,20240331,20240630
0,017837.OF,博时中债7-10年政策性金融债A,博时基金管理有限公司,2023-03-15,NaN,NaN,NaN,NaN,NaN,NaN,2.14,10.56,6.56,7.22,7.32,27.85
1,008054.OF,汇添富中债7-10年国开债A,汇添富基金管理股份有限公司,2020-01-14,0.25,0.26,0.26,0.51,5.32,0.78,0.56,0.58,1.43,9.11,19.54,38.78
2,006961.OF,南方7-10年国开债A,南方基金管理股份有限公司,2019-03-15,1.40,1.82,1.51,1.89,8.18,2.16,1.88,2.66,5.03,4.43,50.34,100.29
3,018266.OF,富国中债7-10年政策性金融债联接A,富国基金管理有限公司,2023-04-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.15,3.66,23.44,10.27,26.42
4,003376.OF,广发7-10年国开行A,广发基金管理有限公司,2016-09-26,50.45,76.21,61.17,43.65,65.54,44.19,38.77,49.31,107.34,114.73,167.74,323.60
5,007228.OF,华安中债7-10年国开行A,华安基金管理有限公司,2019-11-13,6.54,6.36,6.51,5.81,57.10,32.61,39.09,45.91,39.42,32.79,45.23,87.76
6,003358.OF,易方达7-10年国开行A,易方达基金管理有限公司,2016-09-27,55.98,79.09,60.72,46.40,71.88,37.61,30.77,32.90,29.66,36.71,58.19,98.62
7,020248.OF,惠升中债7-10年政金债指数,惠升基金管理有限责任公司,2024-03-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.00,12.61


In [167]:
Range_of_return = pd.DataFrame(scale_data)

In [168]:
fund_scale_df['近三年平均规模'] = fund_scale_df.iloc[:, 4:16].mean(axis=1)
fund_scale_df['近两年平均规模'] = fund_scale_df.iloc[:, 8:16].mean(axis=1)
fund_scale_df['近一年平均规模'] = fund_scale_df.iloc[:, 12:16].mean(axis=1)
index_1 = fund_scale_df.loc[fund_scale_df.iloc[:, 4:16].isnull().any(axis=1)].index
index_2 = fund_scale_df.loc[fund_scale_df.iloc[:, 8:16].isnull().any(axis=1)].index
index_3 = fund_scale_df.loc[fund_scale_df.iloc[:, 12:16].isnull().any(axis=1)].index
fund_scale_df['近三年平均规模'][index_1] = np.nan
fund_scale_df['近两年平均规模'][index_2] = np.nan
fund_scale_df['近一年平均规模'][index_3] = np.nan

fund_scale_df.insert(4, '近三年平均规模', fund_scale_df.pop('近三年平均规模'))
fund_scale_df.insert(5, '近两年平均规模', fund_scale_df.pop('近两年平均规模'))
fund_scale_df.insert(6, '近一年平均规模', fund_scale_df.pop('近一年平均规模'))

<ipython-input-168-4fc9fff83c9d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fund_scale_df['近三年平均规模'][index_1] = np.nan
<ipython-input-168-4fc9fff83c9d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fund_scale_df['近两年平均规模'][index_2] = np.nan
<ipython-input-168-4fc9fff83c9d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fund_scale_df['近一年平均规模'][index_3] = np.nan


In [169]:
grouped = fund_scale_df.groupby('基金公司').agg({'近三年平均规模': 'sum', '近两年平均规模': 'sum', '近一年平均规模': 'sum'}).reset_index()
grouped.columns = ['基金公司', '近三年总规模', '近两年总规模', '近一年总规模', ]
fund_scale_df = fund_scale_df.merge(grouped, on='基金公司')

In [170]:
fund_scale_df['近三年规模权重'] = fund_scale_df['近三年平均规模'] / fund_scale_df['近三年总规模']
fund_scale_df['近两年规模权重'] = fund_scale_df['近两年平均规模'] / fund_scale_df['近两年总规模']
fund_scale_df['近一年规模权重'] = fund_scale_df['近一年平均规模'] / fund_scale_df['近一年总规模']

fund_scale_df.insert(7, '近三年总规模', fund_scale_df.pop('近三年总规模'))
fund_scale_df.insert(8, '近两年总规模', fund_scale_df.pop('近两年总规模'))
fund_scale_df.insert(9, '近一年总规模', fund_scale_df.pop('近一年总规模'))
fund_scale_df.insert(4, '近三年规模权重', fund_scale_df.pop('近三年规模权重'))
fund_scale_df.insert(5, '近两年规模权重', fund_scale_df.pop('近两年规模权重'))
fund_scale_df.insert(6, '近一年规模权重', fund_scale_df.pop('近一年规模权重'))

In [171]:
fund_scale_df.insert(2,'成立日期',fund_scale_df.pop('成立日期'))

In [172]:
fund_scale_df1_cols=['近一年规模权重','近两年规模权重','近三年规模权重','近一年平均规模','近两年平均规模','近三年平均规模','近一年总规模','近两年总规模','近三年总规模',
                '20210930','20211231','20220331','20220630','20220930','20221231','20230331','20230630','20230930','20231231','20240331','20240630']
fund_scale_df1 = fund_scale_df[fund_scale_df1_cols]
result_df = pd.concat([return_df,fund_scale_df1],axis=1)

3.加权计算

In [173]:
fund_performance = fund_scale_df.merge(return_df, on='基金代码')
fund_performance['近三年加权回报'] = fund_performance['近三年规模权重'] * fund_performance['近三年回报']
fund_performance['近两年加权回报'] = fund_performance['近两年规模权重'] * fund_performance['近两年回报']
fund_performance['近一年加权回报'] = fund_performance['近一年规模权重'] * fund_performance['近一年回报']

In [174]:
fund_performance.drop(['近三年规模权重', '近两年规模权重', '基金简称_y', '成立日期_y', '近一年规模权重', '近三年回报', '近两年回报', '近一年回报'] ,axis=1, inplace=True)

In [175]:
fund_performance.drop(['近三年总规模','近两年总规模','近一年总规模','基金公司_y','近三年平均规模','近两年平均规模','近一年平均规模','20210930','20211231','20220331','20220630','20220930','20221231','20230331','20230630','20230930','20231231','20240331','20240630'],axis=1,inplace=True)

In [176]:
fund_performance.reset_index()

,index,基金代码,基金简称_x,成立日期_x,基金公司_x,近三年加权回报,近两年加权回报,近一年加权回报
0,0,017837.OF,博时中债7-10年政策性金融债A,2023-03-15,博时基金管理有限公司,NaN,NaN,7.636506
1,1,008054.OF,汇添富中债7-10年国开债A,2020-01-14,汇添富基金管理股份有限公司,16.745043,11.509790,7.351764
2,2,006961.OF,南方7-10年国开债A,2019-03-15,南方基金管理股份有限公司,20.373718,13.166694,7.309300
3,3,018266.OF,富国中债7-10年政策性金融债联接A,2023-04-07,富国基金管理有限公司,NaN,NaN,7.104953
4,4,003376.OF,广发7-10年国开行A,2016-09-26,广发基金管理有限公司,19.877950,13.082014,7.248418
5,5,007228.OF,华安中债7-10年国开行A,2019-11-13,华安基金管理有限公司,18.713558,12.237465,7.051583
6,6,003358.OF,易方达7-10年国开行A,2016-09-27,易方达基金管理有限公司,18.771401,11.978162,6.740059
7,7,020248.OF,惠升中债7-10年政金债指数,2024-03-13,惠升基金管理有限责任公司,NaN,NaN,NaN


In [177]:
def sum_for_nan(series):
    if series.count() == 0:
        sum_result = np.nan
    else:
        sum_result = series.sum()
    return sum_result

In [178]:
mgrcomp_list = grouped['基金公司'].tolist()
mgrcomp_rank = pd.DataFrame()
for mgrcomp in mgrcomp_list:
    sum_3y = sum_for_nan(fund_performance['近三年加权回报'][fund_performance['基金公司_x'] == mgrcomp])
    sum_2y = sum_for_nan(fund_performance['近两年加权回报'][fund_performance['基金公司_x'] == mgrcomp])
    sum_1y = sum_for_nan(fund_performance['近一年加权回报'][fund_performance['基金公司_x'] == mgrcomp])
    new_row = pd.DataFrame([{'基金公司': mgrcomp, '近三年收益率': sum_3y, '近两年收益率': sum_2y, '近一年收益率': sum_1y, }])
    mgrcomp_rank = pd.concat([mgrcomp_rank, new_row])


In [179]:
mgrcomp_rank['近三年排名'] = mgrcomp_rank['近三年收益率'].rank(method='min', ascending=False)
mgrcomp_rank['近两年排名'] = mgrcomp_rank['近两年收益率'].rank(method='min', ascending=False)
mgrcomp_rank['近一年排名'] = mgrcomp_rank['近一年收益率'].rank(method='min', ascending=False)

In [180]:
mgrcomp_rank.insert(2,'近三年排名',mgrcomp_rank.pop('近三年排名'))
mgrcomp_rank.insert(4,'近两年排名',mgrcomp_rank.pop('近两年排名'))

In [181]:
threshold_3 = mgrcomp_rank['近三年排名'].max() * 0.7
threshold_2 = mgrcomp_rank['近两年排名'].max() * 0.7
threshold_1 = mgrcomp_rank['近一年排名'].max() * 0.7

def calculate_score(row):
    if pd.isna(row['近三年排名']) and pd.isna(row['近两年排名']):
        return '是' if row['近一年排名'] > threshold_1 else ''
    elif pd.isna(row['近三年排名']):
        return '是' if (row['近两年排名'] > threshold_2) & (row['近一年排名'] > threshold_1) else ''
    else:
        return '是' if (row['近三年排名'] > threshold_3) & (row['近两年排名'] > threshold_2) & (row['近一年排名'] > threshold_1) else ''

# 添加"得分是否在后30%"列
mgrcomp_rank['业绩排名是否在后30%'] = mgrcomp_rank.apply(calculate_score, axis=1)

In [182]:
mgrcomp_rank=mgrcomp_rank.sort_values(by='近一年排名', ascending=True)

mgrcomp_rank['近三年收益率'] = mgrcomp_rank['近三年收益率'].round(2)
mgrcomp_rank['近两年收益率'] = mgrcomp_rank['近两年收益率'].round(2)
mgrcomp_rank['近一年收益率'] = mgrcomp_rank['近一年收益率'].round(2)

In [183]:
df_dict = {}
df_dict['产品信息'] = result_df
df_dict['基金公司业绩排名'] = mgrcomp_rank

In [184]:
df_dict

{'产品信息':         基金代码                基金简称       成立日期           基金公司     近一年回报  \
 0  017837.OF    博时中债7-10年政策性金融债A 2023-03-15     博时基金管理有限公司  7.636506   
 1  008054.OF      汇添富中债7-10年国开债A 2020-01-14  汇添富基金管理股份有限公司  7.351764   
 2  006961.OF         南方7-10年国开债A 2019-03-15   南方基金管理股份有限公司  7.309300   
 3  018266.OF  富国中债7-10年政策性金融债联接A 2023-04-07     富国基金管理有限公司  7.104953   
 4  003376.OF         广发7-10年国开行A 2016-09-26     广发基金管理有限公司  7.248418   
 5  007228.OF       华安中债7-10年国开行A 2019-11-13     华安基金管理有限公司  7.051583   
 6  003358.OF        易方达7-10年国开行A 2016-09-27    易方达基金管理有限公司  6.740059   
 7  020248.OF      惠升中债7-10年政金债指数 2024-03-13   惠升基金管理有限责任公司       NaN   
 
        近两年回报      近三年回报  近一年规模权重  近两年规模权重  近三年规模权重  ...  20220331  20220630  \
 0        NaN        NaN      1.0      NaN      NaN  ...       NaN       NaN   
 1  11.509790  16.745043      1.0      1.0      1.0  ...      0.26      0.51   
 2  13.166694  20.373718      1.0      1.0      1.0  ...      1.51      1.89   
 3        NaN

In [185]:
import datetime
from typing import Union

In [186]:
file_path = './output/利率债基7-10.xlsx'  
Exl.generate_excel(file_path, df_dict)
Exl.excel_style_1(file_path)